
# makeATR function

This takes the OHLC4 (mean of open, high, low and close) of the previous "k" (tuning parameter, now chosen to be 15) candles to come up with an estimate of the fluctuation of price. This is pretty helpful for setting profit/loss taking limit prices when trading. Calculated as such, for any time i $OHLC_i = \frac{Open_i + High_i +Low_i + Close_i}{4}$
The mean price at any given T is calculated as:
$$\mu_T = \frac{\sum_{i=T-k}^{T}(OHLC_i)}{k}$$

If T is the current candle, makeATR is calculated as such.
$$ATR = \sqrt{\frac{\sum_{i=T-k}^{T}(OHLC_i - \mu_T)}{k}}$$

In [2]:
import statistics as st
def makeATR(df,k):
    x = []
    df["ATR"] = df["Open"]
    for i,row in df.iterrows():
        x.append(st.mean([row["Open"], row["High"],row["Low"], row["Close"]] ))
        df.at[i,"ATR"] = 0 if len(x) <2 else st.stdev(x)
        if len(x) >k:
            x.pop(0) 
    return df

# Get the data
Pull data from yahoo finance API. Decent amount of data free of charge!

Transform  EUR/USD data (5 minute intervals) in the past 60 days into 9 equal folds. This is to aid in strategy validation. The idea is to formulate a strategy, and then apply the strategy to all the test data sets to come up with an estimate of profitability.

In [3]:
import yfinance as yf
import pandas as pd
from datetime import date
from datetime import timedelta
# Get the data
datas = []
today = date.today()
for i in range(1,10):
    data = yf.download(tickers="EURUSD=X", interval="5m", start = (today - (timedelta(days=i*6))), end= (today - ((timedelta(days=(i-1)*6)))))
    datas.append(makeATR(data,15))


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


# Profit and Loss

The profit is encoded as 1, and loss as 0. From every timestamp, we will find if profit taking price is reached or loss taking price first. The loss/profit limit prices for every candle is dependent on the ATR for each candle. 

The tuning parameters in this calculation are as follows:

1) $Multiplier_{ATR} := \lambda$: This is how much importance is placed on the ATR. Say ATR for EUR/USD is currently 0.0003, and if $\lambda$ is 10 then: $$Profit =Price_T + \lambda \cdot \chi \cdot ATR \implies Price_T + 10 \cdot \chi \cdot 0.0003$$
2) $Ratio_{PL} := \chi$: This is the ratio of how much more profit taking price is than loss. If it is equal to 1.5, then from a specific candle:

    a) $Profit = Price_T + \lambda * 1.5  * ATR$

    b) $Loss = Price_T - \lambda * 1  * ATR$
3) $Timeout := t$ If neither profit or loss is met in $t$ candles, it sells automatically. If $Price_{T+t}$ is lower it's a loss, and 1 if it's higher

Then a column is added where loss (0) or profit (1) is reached. This is required for logistic regression, LDA and QDA. This will help us see the efficacy of trading strategies.

,Open,High,Low,Close,Adj Close,Volume,ATR
Datetime,,,,,,,
2023-02-27 00:00:00+00:00,1.055632,1.055743,1.055298,1.055743,1.055743,0,0.000000
2023-02-27 00:05:00+00:00,1.055298,1.055298,1.054964,1.054964,1.054964,0,0.000335
2023-02-27 00:10:00+00:00,1.054964,1.055520,1.054964,1.055298,1.055298,0,0.000259
2023-02-27 00:15:00+00:00,1.055186,1.055409,1.055186,1.055298,1.055298,0,0.000212
2023-02-27 00:20:00+00:00,1.055186,1.055409,1.055186,1.055298,1.055298,0,0.000184
2023-02-27 00:25:00+00:00,1.055409,1.055855,1.055409,1.055520,1.055520,0,0.000195
2023-02-27 00:30:00+00:00,1.055520,1.055743,1.055409,1.055520,1.055520,0,0.000195
2023-02-27 00:35:00+00:00,1.055409,1.055743,1.055409,1.055409,1.055409,0,0.000186
2023-02-27 00:40:00+00:00,1.055632,1.055632,1.055298,1.055520,1.055520,0,0.000181
